# JSON-Unterstützung in Datenbanken

## JSON - Das Quiz

## JSON-Unterstützung in dokumentenorientierten Datenbanken (am Beispiel von MongoDB)

TODO: MongoDB beschreiben.

TODO: Beispiel beschreiben: Nutzer:innenprofile in einer Stellenbörse.
Wir benutzen dafür die MongoDB-Instanz `demo-mongo`.

### Datenbankverbindungen verwalten

Das Paket `pymongo` steht unter der Apache Software License 2.0.
Es erlaubt die Nutzung von MongoDB in Python.

Die wichtigste Klasse des Pakets heißt `pymongo.MongoClient`.
Sie ermöglicht den Zugriff auf MongoDB-Instanzen.

##### Beispiel

In [1]:
!pip install pymongo~=4.1.1

In [2]:
import pymongo
demo_mongo_client = pymongo.MongoClient("mongodb://demo-mongo")

### Dokumente organisieren

Jede MongoDB-Instanz verwaltet ihre Dokumente in einer dreistufigen Hierarchie:

1. MongoDB-Instanzen enthalten Datenbanken.
2. Datenbanken enthalten Collections.
3. Collections enthalten Dokumente.

Datenbanken und Collections müssen nicht explizit angelegt werden.
Sie werden automatisch angelegt, wenn zum ersten Mal auf sie zugegriffen wird.

##### Beispiel

Wir verwalten die Dokumente unserer Stellenbörse in der Datenbank `employment`:

In [3]:
employment = demo_mongo_client["employment"]
type(employment)

pymongo.database.Database

Die Nutzer:innenprofile speichern wir in der Collection `users`:

In [4]:
users = employment["users"]
type(users)

pymongo.collection.Collection

Zu Beginn unseres Beispiels stellen wir sicher, dass die Collection `users` leer ist: 

In [5]:
users.drop()

### Dokumente einfügen

Die Methode `insert_one` der Klasse `pymongo.collection.Collection` fügt ein JSON-Objekt zur Collection hinzu.

Falls das JSON-Objekt ein Attribut mit dem Namen `_id` enthält, verwendet MongoDB dieses Attribut als Primärschlüssel.

Falls nicht, erzeugt MongoDB einen Primärschlüssel und fügt ihn als Attribut mit dem Namen `_id` hinzu.

In [6]:
users.insert_one({
    "_id": 251,
    "first_name": "Bill",
    "last_name": "Gates",
    "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
    "region": "Greater Seattle Area",
    "industry": "Philanthropy",
    "positions": [
        {"job_title": "Co-chair", "organization": "Bill & Melinda Gates Foundation"},
        {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
    ],
    "education": [
        {"school_name": "Harvard University", "start": 1973, "end": 1975},
        {"school_name": "Lakeside School, Seattle"}
    ],
    "contact_info": {
        "blog": "https://www.gatesnotes.com/",
        "twitter": "https://twitter.com/BillGates"
    }
})

### Dokumenten abfragen

#### Alle Dokumente abfragen

Die Methode `find` der Klasse `pymongo.collection.Collection` liefert einen Cursor als Rückgabewert:

In [7]:
documents = users.find({})
documents

Wir konstruieren eine Liste, um die Ergebnisse der Abfrage zu erhalten:

In [8]:
list(documents)

[{'_id': 251,
  'first_name': 'Bill',
  'last_name': 'Gates',
  'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
  'region': 'Greater Seattle Area',
  'industry': 'Philanthropy',
  'positions': [{'job_title': 'Co-chair',
    'organization': 'Bill & Melinda Gates Foundation'},
   {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
  'education': [{'school_name': 'Harvard University',
    'start': 1973,
    'end': 1975},
   {'school_name': 'Lakeside School, Seattle'}],
  'contact_info': {'blog': 'https://www.gatesnotes.com/',
   'twitter': 'https://twitter.com/BillGates'}}]

#### Felder einschränken

Die Methode `find` nimmt ein zweites Argument entgegen.
Mit diesem Argument weisen wir MongoDB an, nur bestimmte Felder der Dokumente zurückzugeben.

In [9]:
documents = users.find({}, {"first_name": 1, "positions.organization": 1})
list(documents)

[{'_id': 251,
  'first_name': 'Bill',
  'positions': [{'organization': 'Bill & Melinda Gates Foundation'},
   {'organization': 'Microsoft'}]}]

Mit dem Punkt in `positions.organization` konnten wir auf das Attribut `organization` der Objekte im Attribut `positions` zugreifen.
Da es sich bei `positions` um ein Array von Objekten handelt, haben wir auch ein Array von Objekten erhalten (mit dem einizigen Attribut `organization`).

Zusätzlich zu den von uns spezifizierten Attributen `first_name` und `positions.organization` haben wir noch den Primärschlüssel `_id` erhalten.
Wenn wir den Primärschlüssel nicht erhalten wollen, müssen wir ihn explizit ausschließen:

In [10]:
names_only = {"_id": 0, "first_name": 1, "last_name": 1}
documents = users.find({}, names_only)
list(documents)

[{'first_name': 'Bill', 'last_name': 'Gates'}]

#### Dokumente einschränken

Neben den Feldern können wir auch einschränken, welche Dokumente der Collection die Methode `find` zurückgeben soll.

Wenn wir nach einem bestimmten Primärschlüssel suchen, erhalten wir maximal ein Dokument:

In [11]:
documents = users.find({"_id": 251}, names_only)
list(documents)

[{'first_name': 'Bill', 'last_name': 'Gates'}]

In [12]:
documents = users.find({"_id": 12}, names_only)
list(documents)

[]

Neben dem Primärschlüssel können wir auch nach anderen Attributen suchen:

In [13]:
# Alle Dokumente mit dem Attribut last_name="Gates":
documents = users.find({"last_name": "Gates"}, names_only)
list(documents)

[{'first_name': 'Bill', 'last_name': 'Gates'}]

Mit dem Punkt können wir auch nach Attributen von Unterobjekten suchen:

In [14]:
documents = users.find(
    {"contact_info.twitter": "https://twitter.com/BillGates"}
)
list(documents)

[{'_id': 251,
  'first_name': 'Bill',
  'last_name': 'Gates',
  'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
  'region': 'Greater Seattle Area',
  'industry': 'Philanthropy',
  'positions': [{'job_title': 'Co-chair',
    'organization': 'Bill & Melinda Gates Foundation'},
   {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
  'education': [{'school_name': 'Harvard University',
    'start': 1973,
    'end': 1975},
   {'school_name': 'Lakeside School, Seattle'}],
  'contact_info': {'blog': 'https://www.gatesnotes.com/',
   'twitter': 'https://twitter.com/BillGates'}}]

Wenn das Attribut links des Punkts den Typ Array hat, genügt es, wenn eines der Elemente ein Objekt mit dem gewünschten Attribut ist:

In [15]:
documents = users.find(
    {"positions.organization": "Microsoft"},
    {"_id": 0, "first_name": 1, "last_name": 1, "positions": 1}
)
list(documents)

[{'first_name': 'Bill',
  'last_name': 'Gates',
  'positions': [{'job_title': 'Co-chair',
    'organization': 'Bill & Melinda Gates Foundation'},
   {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}]}]

Wir wollen nun alle Dokumente finden, deren Attribut `education`
1. den Typ Array hat und
2. ein oder mehrere bestimmte Elemente enthält.

Dafür benötigen wir den Operator `$all`:

In [16]:
documents = users.find(
    {"education": {"$all": [{"school_name": "Lakeside School, Seattle"}]}},
    {"_id": 0, "first_name": 1, "last_name": 1, "education": 1}
)
list(documents)

[{'first_name': 'Bill',
  'last_name': 'Gates',
  'education': [{'school_name': 'Harvard University',
    'start': 1973,
    'end': 1975},
   {'school_name': 'Lakeside School, Seattle'}]}]

### Dokumente verändern

Die Methode `update_many` der Klasse `pymongo.collection.Collection` ermöglicht das Verändern von Dokumenten:

In [17]:
users.update_many({"_id": 251}, {"$set": {"first_name": "William"}})
documents = users.find({"_id": 251}, names_only)
list(documents)

[{'first_name': 'William', 'last_name': 'Gates'}]

### Dokumente löschen

Die Methode `delete_many` der Klasse `pymongo.collection.Collection` ermöglicht das Löschen von Dokumenten:

In [18]:
users.delete_many({"last_name": "Gates"})
documents = users.find({"_id": 251}, names_only)
list(documents)

[]

### Referenzen zwischen Dokumenten

Mit wachsender Nutzer:innenzahl der Stellenbörse, wollen wir unsere Nutzer:innen-Dokumente normalisieren.
Das heißt, wir wollen
1. einzelne Informationen der Nutzer:innen in eigene Collections auslagern (bspw. die Branche (`industry`)) und
2. das Dokument der jeweiligen Branche im Dokument des:der Nutzer:in referenzieren.

Dafür legen wir die Collection `industries` an:

In [19]:
industries = employment["industries"]
industries.drop()
industries.insert_many([
    {"_id": 43, "name": "Financial Services", "description": "Banking, etc."},
    {"_id": 48, "name": "Construction"},
    {"_id": 131, "name": "Philanthropy"}
])

Außerdem legen wir die Collection `regions` an:

In [20]:
regions = employment["regions"]
regions.drop()
regions.insert_many([
    {"_id": "us:7", "name": "Greater Boston Area"},
    {"_id": "us:91", "name": "Greater Seattle Area", "state": "Washington"}
])

Wir fügen nun einen Nutzer hinzu, dessen Attribute `industry` und `region` auf jeweils ein Dokument der entsprechenden Collection verweisen:

In [21]:
import bson.dbref
users.insert_one({
    "_id": 251,
    "first_name": "Bill",
    "last_name": "Gates",
    "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
    "region": bson.dbref.DBRef(collection=regions.name, id="us:91"),
    "industry": bson.dbref.DBRef(collection=industries.name, id="131"),
    "positions": [
        {"job_title": "Co-chair", "organization": "Bill & Melinda Gates Foundation"},
        {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
    ],
    "education": [
        {"school_name": "Harvard University", "start": 1973, "end": 1975},
        {"school_name": "Lakeside School, Seattle"}
    ],
    "contact_info": {
        "blog": "https://www.gatesnotes.com/",
        "twitter": "https://twitter.com/BillGates"
    }
})

Eine Abfrage dieses Nutzers liefert keinen menschenlesbaren Namen der Region.
Stattdessen wird ein Fremdschlüssel auf die Collection `regions` geliefert:

In [22]:
user_251 = users.find_one({"_id": 251}, {"_id": 0, "last_name": 1, "region": 1})
user_251

{'last_name': 'Gates', 'region': DBRef('regions', 'us:91')}

Dieser Fremdschlüssel muss von der Anwendung aufgelöst werden.
Die Klasse `pymongo.database.Database` stellt dafür die Methode `dereference` bereit:

In [23]:
employment.dereference(user_251["region"])

{'_id': 'us:91', 'name': 'Greater Seattle Area', 'state': 'Washington'}

Wir haben zwei Datenbank-Abfragen benötigt, um den Namen der Region des Nutzers mit dem Primärschlüssel `251` zu finden:
1. Finde das Dokument mit dem Primärschlüssel `251` in der Collection `users`.
2. Finde das Dokument mit dem Primärschlüssel `us:91` in der Collection `regions`.

Relationale Datenbankmanagementsysteme erlauben die Verbindung zweier Tabellen in einer Abfrage mithilfe des `join`-Statements.
Eine vergleichbare Verbindung zweier Collections in einer Abfrage wird von MongoDB nicht unterstützt.

## JSON-Unterstützung in relationalen Datenbanken (am Beispiel von PostgreSQL)

### Datenbankverbindungen verwalten

##### Beispiel

In [24]:
!pip install psycopg[binary]~=3.0.13

In [25]:
import psycopg
conn = psycopg.connect("postgresql://postgres@demo-postgres/postgres", autocommit=True)
cur = conn.cursor()

### Dokumente organisieren

##### Beispiel

In [26]:
cur.execute("drop table if exists users")
cur.execute("create table users(doc jsonb)")

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=demo-postgres database=postgres) at 0x7f8f21825040>

### Dokumente einfügen

In [27]:
user_251 = {
    "_id": 251,
    "first_name": "Bill",
    "last_name": "Gates",
    "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
    "region": "Greater Seattle Area",
    "industry": "Philanthropy",
    "positions": [
        {
            "job_title": "Co-chair",
            "organization": "Bill & Melinda Gates Foundation"
        },
        {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
    ],
    "education": [
        {"school_name": "Harvard University", "start": 1973, "end": 1975},
        {"school_name": "Lakeside School, Seattle"}
    ],
    "contact_info": {
        "blog": "https://www.gatesnotes.com/",
        "twitter": "https://twitter.com/BillGates"
    }
}
import psycopg.types.json
cur.execute(
    "insert into users (doc) values (%s)",
    (psycopg.types.json.Jsonb(user_251),)
)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=demo-postgres database=postgres) at 0x7f8f21825040>

### Dokumente abfragen

#### Alle Dokumente abfragen

In [28]:
cur.execute("select doc from users")
cur.fetchall()

[({'_id': 251,
   'region': 'Greater Seattle Area',
   'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
   'industry': 'Philanthropy',
   'education': [{'end': 1975,
     'start': 1973,
     'school_name': 'Harvard University'},
    {'school_name': 'Lakeside School, Seattle'}],
   'last_name': 'Gates',
   'positions': [{'job_title': 'Co-chair',
     'organization': 'Bill & Melinda Gates Foundation'},
    {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
   'first_name': 'Bill',
   'contact_info': {'blog': 'https://www.gatesnotes.com/',
    'twitter': 'https://twitter.com/BillGates'}},)]

#### Felder einschränken

In [29]:
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization')
    from users
""")
cur.fetchall()

[('Gates', ['Bill & Melinda Gates Foundation', 'Microsoft'])]

#### Dokumente einschränken

In [30]:
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization')
    from users
    where doc['last_name'] = '"Gates"'
""")
cur.fetchall()

[('Gates', ['Bill & Melinda Gates Foundation', 'Microsoft'])]

In [31]:
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization')
    from users
    where doc['last_name'] = '"Jobs"'
""")
cur.fetchall()

[]

In [32]:
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization')
    from users
    where doc['contact_info']['twitter'] = '"https://twitter.com/BillGates"'
""")
cur.fetchall()

[('Gates', ['Bill & Melinda Gates Foundation', 'Microsoft'])]

In [33]:
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization'),
        doc['education']
    from users
    where doc['education'] @> '[{"school_name": "Lakeside School, Seattle"}]'
""")
cur.fetchall()

[('Gates',
  ['Bill & Melinda Gates Foundation', 'Microsoft'],
  [{'end': 1975, 'start': 1973, 'school_name': 'Harvard University'},
   {'school_name': 'Lakeside School, Seattle'}])]

In [34]:
cur.execute("""
    select
        doc['last_name'],
        doc['positions']
    from users
    where doc['positions'] @> '[{"organization": "Microsoft"}]'
""")
cur.fetchall()

[('Gates',
  [{'job_title': 'Co-chair',
    'organization': 'Bill & Melinda Gates Foundation'},
   {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}])]

### Dokumente verändern

In [35]:
cur.execute("""
    update users
    set doc['first_name'] = '"William"'
    where doc['_id'] = '251'
""")
cur.execute("""
    select doc['first_name'], doc['last_name']
    from users
    where doc['_id'] = '251'
""")
cur.fetchall()

[('William', 'Gates')]

### Dokumente löschen

In [36]:
cur.execute("""
    delete from users
    where doc['last_name'] = '"Gates"'
""")
cur.execute("""
    select
        doc['last_name'],
        jsonb_path_query_array(doc, '$.positions[*].organization')
    from users
    where doc['_id'] = '251'
""")
cur.fetchall()

[]

### Referenzen zwischen Dokumenten

In [37]:
cur.execute("drop table if exists industries")
cur.execute("create table industries (id int primary key, doc jsonb)")
cur.executemany(
    "insert into industries (id, doc) values (%s, %s)",
    [
        (43, psycopg.types.json.Jsonb(
            {"name": "Financial Services", "description": "Banking, etc."})),
        (48, psycopg.types.json.Jsonb({"name": "Construction"})),
        (131, psycopg.types.json.Jsonb({"name": "Philanthropy"}))
    ]
)
cur.execute("select id, doc from industries")
cur.fetchall()

[(43, {'name': 'Financial Services', 'description': 'Banking, etc.'}),
 (48, {'name': 'Construction'}),
 (131, {'name': 'Philanthropy'})]

In [38]:
cur.execute("drop table if exists regions")
cur.execute("create table regions (id text primary key, doc jsonb)")
cur.executemany(
    "insert into regions (id, doc) values (%s, %s)",
    [
        ("us:7", psycopg.types.json.Jsonb({"name": "Greater Boston Area"})),
        ("us:91", psycopg.types.json.Jsonb(
            {"name": "Greater Seattle Area", "state": "Washington"}))
    ]
)
cur.execute("select id, doc from regions")
cur.fetchall()

[('us:7', {'name': 'Greater Boston Area'}),
 ('us:91', {'name': 'Greater Seattle Area', 'state': 'Washington'})]

In [39]:
cur.execute("drop table if exists users")
cur.execute("""
    create table users (
        id integer primary key,
        industry_id integer references industries(id),
        region_id text references regions(id),
        doc jsonb
    )
""")
cur.execute(
    "insert into users (id, industry_id, region_id, doc) values (%s, %s, %s, %s)",
    (251, 131, "us:91", psycopg.types.json.Jsonb({
        "first_name": "Bill",
        "last_name": "Gates",
        "summary": "Co-chair of the Bill & Melinda Gates... Active blogger.",
        "positions": [
            {"job_title": "Co-chair",
             "organization": "Bill & Melinda Gates Foundation"},
            {"job_title": "Co-founder, Chairman", "organization": "Microsoft"}
        ],
        "education": [
            {"school_name": "Harvard University", "start": 1973, "end": 1975},
            {"school_name": "Lakeside School, Seattle"}
        ],
        "contact_info": {
            "blog": "https://www.gatesnotes.com/",
            "twitter": "https://twitter.com/BillGates"
        }
    }))
)
cur.execute("select id, industry_id, region_id, doc from users")
cur.fetchall()

[(251,
  131,
  'us:91',
  {'summary': 'Co-chair of the Bill & Melinda Gates... Active blogger.',
   'education': [{'end': 1975,
     'start': 1973,
     'school_name': 'Harvard University'},
    {'school_name': 'Lakeside School, Seattle'}],
   'last_name': 'Gates',
   'positions': [{'job_title': 'Co-chair',
     'organization': 'Bill & Melinda Gates Foundation'},
    {'job_title': 'Co-founder, Chairman', 'organization': 'Microsoft'}],
   'first_name': 'Bill',
   'contact_info': {'blog': 'https://www.gatesnotes.com/',
    'twitter': 'https://twitter.com/BillGates'}})]

In [40]:
cur.execute("""
    select u.doc['last_name'], r.doc['name']
    from users u join regions r on u.region_id = r.id
    where u.id = 251
""")
cur.fetchall()

[('Gates', 'Greater Seattle Area')]

In [41]:
cur.close()
conn.close()

## Zusammenfassung

TODO